# Project: Part of Speech Tagging with Hidden Markov Models 
---
### Introduction

Part of speech tagging is the process of determining the syntactic category of a word from the words in its surrounding context. It is often used to help disambiguate natural language phrases because it can be done quickly with high accuracy. Tagging can be used for many NLP tasks like determining correct pronunciation during speech synthesis (for example, _dis_-count as a noun vs dis-_count_ as a verb), for information retrieval, and for word sense disambiguation.

In this notebook, you'll use the [Pomegranate](http://pomegranate.readthedocs.io/) library to build a hidden Markov model for part of speech tagging using a "universal" tagset. Hidden Markov models have been able to achieve [>96% tag accuracy with larger tagsets on realistic text corpora](http://www.coli.uni-saarland.de/~thorsten/publications/Brants-ANLP00.pdf). Hidden Markov models have also been used for speech recognition and speech generation, machine translation, gene recognition for bioinformatics, and human gesture recognition for computer vision, and more. 

![](_post-hmm.png)

The notebook already contains some code to get you started. You only need to add some new functionality in the areas indicated to complete the project; you will not need to modify the included code beyond what is requested. Sections that begin with **'IMPLEMENTATION'** in the header indicate that you must provide code in the block that follows. Instructions will be provided for each section, and the specifics of the implementation are marked in the code block with a 'TODO' statement. Please be sure to read the instructions carefully!

<div class="alert alert-block alert-info">
**Note:** Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You must then **export the notebook** by running the last cell in the notebook, or by using the menu above and navigating to **File -> Download as -> HTML (.html)** Your submissions should include both the `html` and `ipynb` files.
</div>

<div class="alert alert-block alert-info">
**Note:** Code and Markdown cells can be executed using the `Shift + Enter` keyboard shortcut. Markdown cells can be edited by double-clicking the cell to enter edit mode.
</div>

### The Road Ahead
You must complete Steps 1-3 below to pass the project. The section on Step 4 includes references & resources you can use to further explore HMM taggers.

- [Step 1](#Step-1:-Read-and-preprocess-the-dataset): Review the provided interface to load and access the text corpus
- [Step 2](#Step-2:-Build-a-Most-Frequent-Class-tagger): Build a Most Frequent Class tagger to use as a baseline
- [Step 3](#Step-3:-Build-an-HMM-tagger): Build an HMM Part of Speech tagger and compare to the MFC baseline
- [Step 4](#Step-4:-[Optional]-Improving-model-performance): (Optional) Improve the HMM tagger

In [1]:
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

## Step 1: Read and preprocess the dataset

In [3]:
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

print("There are {} sentences in the corpus.".format(len(data)))
print("There are {} sentences in the training set.".format(len(data.training_set)))
print("There are {} sentences in the testing set.".format(len(data.testing_set)))



There are 57340 sentences in the corpus.
There are 45872 sentences in the training set.
There are 11468 sentences in the testing set.


#### Sentences

`Dataset.sentences` is a dictionary of all sentences in the training corpus, each keyed to a unique sentence identifier. Each `Sentence` is itself an object with two attributes: a tuple of the words in the sentence named `words` and a tuple of the tag corresponding to each word named `tags`.

In [4]:
key = 'b100-38532'
print("Sentence: {}".format(key))
print("words:\n\t{!s}".format(data.sentences[key].words))
print("tags:\n\t{!s}".format(data.sentences[key].tags))

Sentence: b100-38532
words:
	('Perhaps', 'it', 'was', 'right', ';', ';')
tags:
	('ADV', 'PRON', 'VERB', 'ADJ', '.', '.')


In [5]:
print("There are a total of {} samples of {} unique words in the corpus."
      .format(data.N, len(data.vocab)))
print("There are {} samples of {} unique words in the training set."
      .format(data.training_set.N, len(data.training_set.vocab)))
print("There are {} samples of {} unique words in the testing set."
      .format(data.testing_set.N, len(data.testing_set.vocab)))
print("There are {} words in the test set that are missing in the training set."
      .format(len(data.testing_set.vocab - data.training_set.vocab)))


There are a total of 1161192 samples of 56057 unique words in the corpus.
There are 928458 samples of 50536 unique words in the training set.
There are 232734 samples of 25112 unique words in the testing set.
There are 5521 words in the test set that are missing in the training set.


#### Accessing word and tag Sequences
The `Dataset.X` and `Dataset.Y` attributes provide access to ordered collections of matching word and tag sequences for each sentence in the dataset.

In [6]:
# accessing words with Dataset.X and tags with Dataset.Y 
for i in range(2):    
    print("Sentence {}:".format(i + 1), data.X[i])
    print()
    print("Labels {}:".format(i + 1), data.Y[i])
    print()

Sentence 1: ('Mr.', 'Podger', 'had', 'thanked', 'him', 'gravely', ',', 'and', 'now', 'he', 'made', 'use', 'of', 'the', 'advice', '.')

Labels 1: ('NOUN', 'NOUN', 'VERB', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADV', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', '.')

Sentence 2: ('But', 'there', 'seemed', 'to', 'be', 'some', 'difference', 'of', 'opinion', 'as', 'to', 'how', 'far', 'the', 'board', 'should', 'go', ',', 'and', 'whose', 'advice', 'it', 'should', 'follow', '.')

Labels 2: ('CONJ', 'PRT', 'VERB', 'PRT', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADV', 'ADV', 'DET', 'NOUN', 'VERB', 'VERB', '.', 'CONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'VERB', '.')



#### Accessing (word, tag) Samples
The `Dataset.stream()` method returns an iterator that chains together every pair of (word, tag) entries across all sentences in the entire corpus.

In [7]:
print("\nStream (word, tag) pairs:\n")
for i, pair in enumerate(data.stream()):
    print("\t", pair)
    if i > 5: break


Stream (word, tag) pairs:

	 ('Mr.', 'NOUN')
	 ('Podger', 'NOUN')
	 ('had', 'VERB')
	 ('thanked', 'VERB')
	 ('him', 'PRON')
	 ('gravely', 'ADV')
	 (',', '.')


## Step 2: Build a Most Frequent Class tagger

In [8]:
def pair_counts(sequences_A, sequences_B):
    """Returns a dictionary keyed to each unique value in the first sequence list
    that counts the number of occurrences of the corresponding value from the
    second sequences list.
    
    For example, if sequences_A is tags and sequences_B is the corresponding
    words, then if 1244 sequences contain the word "time" tagged as a NOUN, then
    function returns a dictionary such that pair_counts[NOUN][time] == 1244
    """
    
    
    dictu = defaultdict(dict)
    for tag, word in zip(sequences_A, sequences_B):
        
        
        if word not in dictu[tag].keys():
            dictu[tag][word] = 0
        dictu[tag][word] += 1
    
    return dictu

    raise NotImplementedError



alltags,allwords = [],[]
# for i in range(len(data)):
#     for tag,word in zip(data.Y[i],data.X[i]):
#         alltags.append(tag)
#         allwords.append(word)

for pair in data.stream():
    alltags.append(pair[1])
    allwords.append(pair[0])
        


emission_counts = pair_counts(alltags,allwords)




In [9]:
print(emission_counts.keys())

dict_keys(['NOUN', 'VERB', 'PRON', 'ADV', '.', 'CONJ', 'ADP', 'DET', 'PRT', 'ADJ', 'X', 'NUM'])


In [10]:
# Create a lookup table mfc_table where mfc_table[word] contains the tag label most frequently assigned to that word
from collections import namedtuple

FakeState = namedtuple("FakeState", "name")

class MFCTagger:
    
    missing = FakeState(name="<MISSING>")
    
    def __init__(self, table):
        self.table = defaultdict(lambda: MFCTagger.missing)
        self.table.update({word: FakeState(name=tag) for word, tag in table.items()})
        
    def viterbi(self, seq):
        return 0., list(enumerate(["<start>"] + [self.table[w] for w in seq] + ["<end>"]))




train_tags,train_words = [],[]

for pair in data.training_set.stream():
    train_tags.append(pair[1])
    train_words.append(pair[0])
    
    
word_counts = pair_counts(train_tags,train_words)

mfc_table = {}
word_max = {}
for tag in word_counts.keys():
    for word in word_counts[tag].keys():
        if word not in word_max:
            word_max[word] = 0
            
        if word_counts[tag][word]>word_max[word]:
                word_max[word] = word_counts[tag][word]
                mfc_table[word] = tag



mfc_model = MFCTagger(mfc_table) 


### Making Predictions with a Model


In [11]:
def replace_unknown(sequence):
    """Returns a copy of the input sequence where each unknown word is replaced
    by the literal string value 'nan'. 
    """
    return [w if w in data.training_set.vocab else 'nan' for w in sequence]

def simplify_decoding(X, model):
    """X should be a 1-D sequence of observations for the model to predict"""
    _, state_path = model.viterbi(replace_unknown(X))
    return [state[1].name for state in state_path[1:-1]]  

### Example Decoding Sequences with MFC Tagger

In [12]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, mfc_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', '<MISSING>', 'ADP', 'ADJ', 'ADJ', 

### Evaluating Model Accuracy

The function below will evaluate the accuracy of the MFC tagger on the collection of all sentences from a text corpus. 

In [13]:
def accuracy(X, Y, model):
    """Calculates the prediction accuracy by using the model to decode each sequence
    in the input X and comparing the prediction with the true labels in Y.
    
    The X should be an array whose first dimension is the number of sentences to test,
    and each element of the array should be an iterable of the words in the sequence.
    The arrays X and Y should have the exact same shape.
    
    X = [("See", "Spot", "run"), ("Run", "Spot", "run", "fast"), ...]
    Y = [(), (), ...]
    """
    correct = total_predictions = 0
    for observations, actual_tags in zip(X, Y):
        
        try:
            most_likely_tags = simplify_decoding(observations, model)
            correct += sum(p == t for p, t in zip(most_likely_tags, actual_tags))
        except:
            pass
        total_predictions += len(observations)
    return correct / total_predictions

#### Evaluate the accuracy of the MFC tagger

In [14]:
mfc_training_acc = accuracy(data.training_set.X, data.training_set.Y, mfc_model)
print("training accuracy mfc_model: {:.2f}%".format(100 * mfc_training_acc))

mfc_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, mfc_model)
print("testing accuracy mfc_model: {:.2f}%".format(100 * mfc_testing_acc))


training accuracy mfc_model: 95.72%
testing accuracy mfc_model: 93.02%


## Step 3: Build an HMM tagger


### Unigram Counts

Function below estimates the co-occurrence frequency of each symbol over all of the input sequences. The unigram probabilities in our HMM model are estimated from the formula below, where N is the total number of samples in the input.

$$P(tag_1) = \frac{C(tag_1)}{N}$$

In [15]:
def unigram_counts(sequences):
    """Returns a dictionary keyed to each unique value in the input sequence list that
    counts the number of occurrences of the value in the sequences list. The sequences
    collection should be a 2-dimensional array.
    
    For example, if the tag NOUN appears 275558 times over all the input sequences,
    then function returns a dictionary such that your_unigram_counts[NOUN] == 275558.
    """
    
    unidict = {}
    for i in range(len(sequences)):
        for tag in sequences[i]:
            if tag not in unidict.keys():
                unidict[tag]=0
            unidict[tag]+=1
    
    return unidict
    raise NotImplementedError


tag_unigrams = unigram_counts(data.training_set.Y) 



In [16]:
print(tag_unigrams)

{'ADV': 44877, 'NOUN': 220632, '.': 117757, 'VERB': 146161, 'ADP': 115808, 'ADJ': 66754, 'CONJ': 30537, 'DET': 109671, 'PRT': 23906, 'NUM': 11878, 'PRON': 39383, 'X': 1094}


### Bigram Counts

Function below to estimates the co-occurrence frequency of each pair of symbols in each of the input sequences. These counts are used in the HMM model to estimate the bigram probability of two tags from the frequency counts according to the formula: $$P(tag_2|tag_1) = \frac{C(tag_2|tag_1)}{C(tag_2)}$$


In [17]:
def bigram_counts(sequences):
    """Returns a dictionary keyed to each unique PAIR of values in the input sequences
    list that counts the number of occurrences of pair in the sequences list. The input
    should be a 2-dimensional array.
    
    For example, if the pair of tags (NOUN, VERB) appear 61582 times, then function
    returns a dictionary such that your_bigram_counts[(NOUN, VERB)] == 61582
    """
    bidict = {}
    for seq in sequences:
        for i in range(len(seq)-1):
            
            if (seq[i],seq[i+1]) not in bidict.keys():
                bidict[(seq[i],seq[i+1])] = 0
            bidict[(seq[i],seq[i+1])] +=1

    
    return bidict               

    
    raise NotImplementedError


tag_bigrams = bigram_counts(data.training_set.Y)




In [18]:
print(tag_bigrams)

{('ADV', 'NOUN'): 1478, ('NOUN', '.'): 62639, ('.', 'ADV'): 5124, ('ADV', '.'): 7577, ('.', 'VERB'): 9041, ('VERB', 'ADP'): 24927, ('ADP', 'ADJ'): 9533, ('ADJ', 'NOUN'): 43664, ('NOUN', 'CONJ'): 13185, ('CONJ', 'VERB'): 6012, ('VERB', 'ADJ'): 8423, ('.', 'DET'): 8008, ('DET', 'VERB'): 7062, ('ADJ', 'PRT'): 1301, ('PRT', 'ADP'): 2189, ('ADP', 'NUM'): 3467, ('NUM', 'NOUN'): 4524, ('.', 'PRON'): 5448, ('PRON', 'VERB'): 27860, ('VERB', 'PRT'): 9556, ('PRT', 'VERB'): 14886, ('VERB', 'NOUN'): 14230, ('NOUN', 'NUM'): 1783, ('NUM', '.'): 3210, ('.', 'NUM'): 1412, ('.', '.'): 12588, ('ADP', 'ADV'): 1805, ('ADV', 'NUM'): 597, ('DET', 'NOUN'): 68785, ('CONJ', 'DET'): 4636, ('NOUN', 'VERB'): 34972, ('ADP', 'NOUN'): 29965, ('ADP', 'DET'): 52841, ('NOUN', 'ADP'): 53884, ('CONJ', 'NOUN'): 7502, ('.', 'NOUN'): 9782, ('VERB', '.'): 11699, ('VERB', 'VERB'): 26957, ('.', 'ADP'): 7595, ('ADV', 'DET'): 3309, ('DET', 'ADJ'): 26236, ('NOUN', 'DET'): 3425, ('ADJ', '.'): 6666, ('VERB', 'DET'): 23764, ('ADJ', '

### Sequence Starting Counts
Functions below to estimates the bigram probabilities of a sequence starting with each tag.

In [19]:
def starting_counts(sequences):
    """Returns a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the beginning of
    a sequence.
    
    For example, if 8093 sequences start with NOUN, then function returns a
    dictionary such that your_starting_counts[NOUN] == 8093
    """
    
    startcount_dict = {}
    for seq in sequences:
        if seq[0] not in startcount_dict:
            startcount_dict[seq[0]]=0
        startcount_dict[seq[0]]+=1
    return startcount_dict
    raise NotImplementedError


tag_starts = starting_counts(data.training_set.Y)



In [20]:
print(tag_starts)

{'ADV': 4185, 'ADP': 5583, 'ADJ': 1582, 'PRT': 1718, 'DET': 9763, 'PRON': 7318, 'NOUN': 6469, 'CONJ': 2282, '.': 4107, 'NUM': 760, 'VERB': 2080, 'X': 25}


### Sequence Ending Counts
Function below to estimates the bigram probabilities of a sequence ending with each tag.

In [21]:
def ending_counts(sequences):
    """Returns a dictionary keyed to each unique value in the input sequences list
    that counts the number of occurrences where that value is at the end of
    a sequence.
    
    For example, if 18 sequences end with DET, then function returns a
    dictionary such that ending_counts[DET] == 18
    """
    
    
    endcount_dict = {}
    for seq in sequences:
        if seq[-1] not in endcount_dict:
            endcount_dict[seq[-1]]=0
        endcount_dict[seq[-1]]+=1
    return endcount_dict
    raise NotImplementedError


tag_ends = ending_counts(data.training_set.Y)



In [22]:
print(tag_ends)

{'.': 44936, 'NOUN': 722, 'NUM': 63, 'VERB': 75, 'ADJ': 25, 'ADV': 16, 'ADP': 7, 'DET': 14, 'CONJ': 2, 'PRON': 4, 'PRT': 7, 'X': 1}


### Basic HMM Tagger
The unigrams and bigrams calculated is used to construct a hidden Markov tagger.

- Add one state per tag
    - The emission distribution at each state should be estimated with the formula: $P(w|t) = \frac{C(t, w)}{C(t)}$
- Add an edge from the starting state `basic_model.start` to each tag
    - The transition probability should be estimated with the formula: $P(t|start) = \frac{C(start, t)}{C(start)}$
- Add an edge from each tag to the end state `basic_model.end`
    - The transition probability should be estimated with the formula: $P(end|t) = \frac{C(t, end)}{C(t)}$
- Add an edge between _every_ pair of tags
    - The transition probability should be estimated with the formula: $P(t_2|t_1) = \frac{C(t_1, t_2)}{C(t_1)}$

In [ ]:
basic_model = HiddenMarkovModel(name="base-hmm-tagger")
basic_model.states = []


state_dict = {}

traintags, trainwords = [], []

for pair in data.stream():
    traintags.append(pair[1])
    trainwords.append(pair[0])

emission_probabilities_all = pair_counts(traintags, trainwords)

for tag in data.training_set.tagset:
    emission_probabilities = emission_probabilities_all[tag]
    for word in emission_probabilities:
        emission_probabilities[word] = emission_probabilities[word] / tag_unigrams[tag]
    emission_distribution = DiscreteDistribution(emission_probabilities)
    state = State(emission_distribution, name=tag)
    state_dict[tag]=state
    basic_model.states.append(state)
    



for tag in data.training_set.tagset:
    tag_starts[tag] = tag_starts[tag]/len(data.training_set.Y)
      
    tag_ends[tag] = tag_ends[tag]/len(data.training_set.Y) 

for tag in data.training_set.tagset:
    basic_model.add_transition(basic_model.start, state_dict[tag], tag_starts[tag])
    basic_model.add_transition(state_dict[tag], basic_model.end, tag_ends[tag])



for tag1 in data.training_set.tagset:
    for tag2 in data.training_set.tagset:
        transition_prob = tag_bigrams[(tag1, tag2)] / tag_unigrams[tag1]
        basic_model.add_transition(state_dict[tag1], state_dict[tag2], transition_prob)


basic_model.bake()        




In [24]:
hmm_training_acc = accuracy(data.training_set.X, data.training_set.Y, basic_model)
print("training accuracy basic hmm model: {:.2f}%".format(100 * hmm_training_acc))

hmm_testing_acc = accuracy(data.testing_set.X, data.testing_set.Y, basic_model)
print("testing accuracy basic hmm model: {:.2f}%".format(100 * hmm_testing_acc))



training accuracy basic hmm model: 97.54%
testing accuracy basic hmm model: 96.18%


### Example Decoding Sequences with the HMM Tagger

In [25]:
for key in data.testing_set.keys[:3]:
    print("Sentence Key: {}\n".format(key))
    print("Predicted labels:\n-----------------")
    print(simplify_decoding(data.sentences[key].words, basic_model))
    print()
    print("Actual labels:\n--------------")
    print(data.sentences[key].tags)
    print("\n")

Sentence Key: b100-28144

Predicted labels:
-----------------
['CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.']

Actual labels:
--------------
('CONJ', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'NOUN', 'NUM', '.', 'CONJ', 'NOUN', 'NUM', '.', '.', 'NOUN', '.', '.')


Sentence Key: b100-23146

Predicted labels:
-----------------
['PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.']

Actual labels:
--------------
('PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'NOUN', 'VERB', 'VERB', '.', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', '.')


Sentence Key: b100-35462

Predicted labels:
-----------------
['DET', 'ADJ', 'NOUN', 'VERB', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', '.', 'ADP', 'ADJ', 'NOUN', '.', 'CONJ', 'ADP', 'DET', 'NOUN', 'ADP', 'ADJ', 'ADJ', '.', 